In [4]:
import numpy as np
import pandas as pd
import pickle

import os
import collections

## Saving Binary format to text using gensim. Reduction of 300 dimensional did not work
### total_vec argument also specified as 100 or 10 is not reducing the dimension.

### Right now it is converted into text format sucessfuly with 300 dimensions

In [2]:
dirname = os.getcwd()
dirname = os.path.dirname(dirname)
dataset_path = os.path.join(dirname, 'datasets/')
print(dataset_path)

/media/kandy/hdd/master-thesis/constituency-parsing/datasets/


In [3]:
from gensim.models.keyedvectors import KeyedVectors

model = KeyedVectors.load_word2vec_format(dataset_path+'GoogleNews-vectors-negative300.bin', binary=True)
model.save_word2vec_format(dataset_path+'GoogleNews-vectors-negative-orig300.txt', fvocab=dataset_path+'/google-vocab.txt', binary=False)

In [4]:
# failed attempts

#model.save_word2vec_format(dataset_path+'GoogleNews-vectors-negative100.txt', binary=False, total_vec=100)
#model.save_word2vec_format(dataset_path+'GoogleNews-vectors-negative10.txt', binary=False, total_vec=10)

## Read_csv chunking and using specific columns are also not working
### skip rows worked smoothly
### chunking - memory consumption was less. But later on it raised up and came down (could not understand). When about to store the chunked data , memory got overflowed in the middle
### using specific columns did not reduce memory consumption (reading 11 and 301 columns are the same)

#googleVectors =  pd.read_csv(dataset_path+'/GoogleNews-vectors-negative-orig300.txt', sep=' ', header=None, skiprows=[0],
#                            usecols=[0,1,2,3,4,5,6,7,8,9,10])

#print(googleVectors.head())

#print(googleVectors.shape) #(2168995, 11)

#toy_google_vectors = googleVectors.iloc[:, 0:11]
#print(toy_google_vectors.shape)
#toy_google_vectors.to_csv(dataset_path+'GoogleNews-vectors-negative10.txt',header=False, index=False,sep=' ')

#reader =  pd.read_csv(dataset_path+'GoogleNews-vectors-negative-orig300.txt', sep=' ', header=None, skiprows=[0],
#                            usecols=[0,1,2,3,4,5,6,7,8,9,10], chunksize=50000)

#j=0
#for chunk in reader:
#    print(chunk.shape)
#    j = j+1
#    chunk.to_csv(dataset_path+'GoogleNews-vectors-negative10_'+str(j)+'.txt',header=False, index=False,sep=' ')

### Use basic File reading and writing. Skipped lines unnecessary
#### split the lines and take only first 11 or 101 dimensional vector (word and vector values)
#### Select the words which are present atleast once or twice in Wall street journal dataset (this immensely reduces the memory)

# Get the frequency from wallstreet journal dataset

In [10]:
with open(dataset_path+'WJ-preprocessing.txt','rb') as pickle_file:
    frequency_words_dict = pickle.load(pickle_file)
print(len(frequency_words_dict))
frequency_words_dict

44377


OrderedDict([(',', 60484),
             ('the', 59459),
             ('.', 48144),
             ('of', 28440),
             ('to', 27448),
             ('a', 24781),
             ('in', 21047),
             ('and', 20449),
             ('*-1', 14220),
             ('0', 12447),
             ('*', 12172),
             ('*t*-1', 11612),
             ("'s", 11556),
             ('for', 10454),
             ('that', 10422),
             ('*u*', 9201),
             ('$', 8817),
             ('``', 8735),
             ('is', 8539),
             ("''", 8506),
             ('it', 7195),
             ('said', 7141),
             ('on', 6646),
             ('%', 6121),
             ('at', 5770),
             ('by', 5705),
             ('as', 5701),
             ('from', 5438),
             ('with', 5356),
             ('*-2', 5350),
             ('million', 5335),
             ('*t*-2', 5030),
             ('mr.', 4950),
             ('was', 4901),
             ('be', 4586),
             ('its',

# Get the words from Google dataset

In [11]:
google_word_list = []
with open(dataset_path+'GoogleNews-vectors-negative-orig300.txt', 'r') as read:
    contents = read.readlines(5)
    while(len(contents) != 0):
        contents = read.readlines(30000)
        contents = [content.split()[0].lower() for content in contents]
        google_word_list.extend(contents)
print(len(google_word_list))
print(google_word_list[3])

3000000
that


## Extraction of data from 3 million rows is time consuming
### Get the indexes from google corpus and group them in batches. This is convenient to extract the content. We can read in batches rather than reading individual lines

In [12]:
words_index = []
for word in frequency_words_dict.keys():
    if word in google_word_list:
        index = google_word_list.index(word)
        words_index.append(index)

print(len(words_index))

31460


In [52]:
sorted_word_index = sorted(words_index)
group_indexes = []
start_index = sorted_word_index[0]
stop_index = sorted_word_index[0]

for index in sorted_word_index[1:]:
    if(index - stop_index) == 1:
        stop_index = index
    else:
        group_indexes.append((start_index, stop_index))
        start_index = index
        stop_index = index

group_indexes.append((start_index, stop_index))
print(len(group_indexes))
group_indexes

18883


[(1, 5),
 (7, 10),
 (12, 24),
 (26, 39),
 (41, 50),
 (52, 56),
 (58, 61),
 (63, 69),
 (71, 83),
 (85, 97),
 (99, 104),
 (106, 122),
 (124, 127),
 (129, 147),
 (150, 155),
 (157, 166),
 (168, 173),
 (175, 175),
 (177, 184),
 (186, 191),
 (193, 205),
 (207, 217),
 (219, 224),
 (226, 227),
 (229, 231),
 (233, 259),
 (261, 267),
 (269, 274),
 (276, 298),
 (301, 304),
 (306, 309),
 (311, 311),
 (313, 315),
 (317, 323),
 (325, 331),
 (333, 354),
 (357, 360),
 (362, 399),
 (401, 408),
 (410, 426),
 (428, 429),
 (432, 441),
 (443, 445),
 (447, 452),
 (454, 469),
 (471, 482),
 (484, 493),
 (495, 529),
 (531, 544),
 (546, 550),
 (552, 552),
 (554, 568),
 (570, 589),
 (591, 616),
 (618, 621),
 (623, 658),
 (660, 664),
 (666, 681),
 (683, 687),
 (689, 707),
 (709, 712),
 (714, 716),
 (718, 718),
 (720, 740),
 (742, 760),
 (762, 789),
 (791, 795),
 (797, 832),
 (834, 846),
 (848, 855),
 (857, 885),
 (887, 892),
 (894, 905),
 (907, 926),
 (928, 938),
 (940, 943),
 (945, 969),
 (971, 971),
 (973, 974

In [63]:
extract_lines = 0
words_extracted = []
i = 0
with open(dataset_path+'GoogleNews-vectors-negative-orig300.txt', 'r') as read:
    with open(dataset_path+'GoogleNews-vectors-negative300.txt', 'w') as write:
        # read the first 5 lines which are headers
        contents = read.readlines(5)
        #print('headers:', contents)
        contents = read.readlines(1)
        write.writelines(contents) # it reads the UNK symbol
        #print('firstline:', contents)
        googleIndex = 1
        for grp in group_indexes:
            # get the group - start and stop indexes (inclusive)
            start = grp[0]
            stop = grp[1]
            
            print(start, stop)
            # when the google index is behind the start - read untill that.
            # read in multiples of 30K if neessary. 
            while(googleIndex < start):
                if((start - googleIndex) > 30000):
                    contents = read.readlines(30000)
                    #print(len(contents), googleIndex, start, stop)
                    googleIndex = googleIndex + len(contents)
                else:
                    contents = read.readlines(start - googleIndex)
                    googleIndex = googleIndex + len(contents)
                    #print(len(contents), googleIndex, start, stop)
            while(googleIndex <= stop):
                # it is a good assumption that the range will not be more than 30000 :-)
                contents = read.readlines(stop - start + 1) # +1 reads including the stop index
                #print(contents)
                contents = [content.split()[0:301] for content in contents]

                words_extracted.extend([content[0].lower() for content in contents])

                contents = [content[0].lower()+' '+' '.join(content[1:])+'\n' for content in contents]

                extract_lines = extract_lines + len(contents)
                write.writelines(contents)
                #print(contents)

                googleIndex = googleIndex + len(contents)
            i = i + 1
            #if (i == 5):
            #    break
            #print(len(contents), googleIndex, start, stop)
        UNK = '</s>'
        FILLER = '</fill>'
        fill_list = [FILLER]
        for i in range(300):
            fill_list.append(str(i))
        content = ' '.join(fill_list) + '\n'
        write.write(content)
            
print(extract_lines)
print(len(words_extracted))
print(words_extracted)

1 5
7 10
12 24
26 39
41 50
52 56
58 61
63 69
71 83
85 97
99 104
106 122
124 127
129 147
150 155
157 166
168 173
175 175
177 184
186 191
193 205
207 217
219 224
226 227
229 231
233 259
261 267
269 274
276 298
301 304
306 309
311 311
313 315
317 323
325 331
333 354
357 360
362 399
401 408
410 426
428 429
432 441
443 445
447 452
454 469
471 482
484 493
495 529
531 544
546 550
552 552
554 568
570 589
591 616
618 621
623 658
660 664
666 681
683 687
689 707
709 712
714 716
718 718
720 740
742 760
762 789
791 795
797 832
834 846
848 855
857 885
887 892
894 905
907 926
928 938
940 943
945 969
971 971
973 974
976 977
979 995
997 1007
1009 1027
1029 1030
1032 1035
1037 1041
1043 1054
1056 1059
1061 1102
1105 1107
1109 1117
1119 1129
1132 1148
1150 1150
1152 1155
1157 1159
1161 1173
1175 1182
1184 1189
1191 1194
1196 1213
1215 1217
1219 1229
1231 1243
1245 1252
1254 1283
1285 1293
1295 1309
1311 1319
1321 1337
1339 1340
1342 1345
1347 1351
1353 1370
1372 1375
1377 1383
1385 1387
1389 1398
1400 14

8098 8100
8102 8109
8111 8112
8114 8116
8119 8119
8122 8128
8130 8132
8134 8134
8136 8142
8144 8148
8150 8158
8161 8165
8167 8168
8170 8171
8173 8175
8179 8183
8185 8189
8192 8192
8194 8194
8197 8198
8200 8201
8203 8206
8208 8210
8212 8218
8220 8224
8226 8226
8228 8231
8233 8236
8239 8239
8241 8241
8243 8246
8249 8255
8258 8259
8261 8265
8267 8268
8270 8274
8276 8280
8282 8282
8285 8285
8288 8289
8292 8296
8298 8302
8304 8306
8308 8308
8311 8330
8332 8332
8334 8334
8336 8340
8343 8344
8347 8350
8352 8353
8355 8356
8358 8358
8360 8360
8362 8363
8365 8366
8370 8377
8380 8386
8388 8389
8391 8393
8395 8396
8398 8398
8400 8400
8402 8407
8409 8415
8419 8424
8428 8429
8431 8433
8435 8438
8440 8440
8443 8443
8446 8450
8452 8454
8457 8463
8465 8465
8472 8473
8475 8475
8477 8481
8483 8488
8492 8493
8496 8499
8502 8502
8504 8509
8511 8512
8514 8514
8517 8523
8528 8531
8533 8538
8540 8540
8542 8544
8546 8547
8550 8551
8553 8559
8561 8562
8564 8567
8569 8569
8571 8571
8574 8580
8582 8586
8588 8589


11603 11605
11607 11614
11616 11620
11622 11622
11625 11626
11628 11628
11630 11630
11632 11632
11634 11634
11638 11639
11641 11645
11647 11648
11650 11655
11659 11661
11663 11668
11670 11671
11674 11676
11681 11685
11687 11691
11693 11693
11695 11706
11715 11718
11720 11724
11728 11729
11732 11734
11736 11736
11741 11743
11745 11745
11755 11756
11758 11760
11763 11763
11765 11768
11770 11772
11775 11779
11781 11784
11786 11787
11789 11789
11791 11797
11801 11801
11803 11803
11805 11806
11809 11809
11812 11812
11814 11814
11817 11817
11820 11820
11823 11823
11825 11825
11829 11835
11838 11839
11842 11842
11844 11844
11849 11851
11853 11853
11858 11862
11864 11867
11869 11869
11871 11871
11874 11875
11878 11880
11882 11882
11884 11885
11887 11894
11897 11897
11901 11908
11910 11911
11914 11917
11921 11924
11930 11930
11932 11932
11934 11937
11939 11939
11941 11942
11946 11949
11952 11957
11959 11960
11963 11963
11968 11971
11974 11976
11978 11981
11984 11984
11988 11990
11992 11998
1200

15011 15012
15016 15017
15019 15019
15022 15022
15024 15024
15026 15027
15029 15030
15032 15035
15037 15037
15039 15041
15043 15043
15050 15057
15061 15061
15063 15068
15070 15071
15073 15073
15077 15078
15081 15082
15084 15084
15086 15086
15088 15091
15094 15099
15101 15102
15105 15105
15108 15111
15114 15115
15117 15119
15121 15121
15124 15124
15126 15127
15134 15134
15136 15136
15139 15140
15144 15149
15152 15154
15156 15157
15160 15160
15163 15163
15167 15170
15174 15174
15177 15177
15181 15181
15183 15184
15187 15192
15194 15195
15198 15198
15202 15203
15205 15205
15207 15207
15209 15210
15215 15218
15220 15220
15223 15226
15229 15229
15232 15232
15234 15235
15237 15237
15241 15246
15249 15249
15251 15251
15254 15254
15258 15260
15262 15262
15264 15265
15267 15267
15269 15270
15272 15274
15276 15276
15278 15279
15282 15284
15287 15288
15290 15290
15292 15292
15295 15295
15297 15299
15301 15302
15304 15304
15306 15306
15308 15308
15310 15312
15314 15314
15318 15322
15325 15326
1532

17887 17889
17894 17894
17896 17896
17899 17899
17901 17902
17904 17904
17908 17908
17910 17912
17915 17915
17917 17917
17920 17920
17925 17925
17928 17928
17937 17937
17941 17941
17945 17945
17947 17947
17949 17949
17952 17952
17954 17954
17958 17959
17961 17961
17966 17969
17973 17973
17975 17975
17977 17979
17982 17982
17984 17986
17989 17989
17991 17991
17993 17993
17995 17995
17997 17997
18003 18005
18008 18010
18012 18012
18014 18014
18017 18019
18022 18024
18030 18032
18040 18040
18042 18042
18047 18048
18051 18051
18054 18054
18056 18056
18059 18059
18062 18062
18065 18065
18067 18068
18070 18070
18073 18075
18078 18078
18081 18081
18083 18085
18088 18088
18090 18090
18092 18092
18097 18097
18099 18100
18102 18102
18104 18105
18108 18108
18110 18111
18113 18113
18117 18118
18120 18122
18124 18124
18126 18126
18128 18129
18131 18131
18134 18135
18138 18138
18141 18141
18146 18153
18155 18155
18159 18162
18164 18169
18172 18173
18178 18179
18181 18187
18190 18191
18193 18194
1819

...

237716 237716
237747 237748
237776 237776
237846 237846
237928 237928
238086 238086
238419 238419
238481 238481
238536 238536
238579 238579
238600 238600
238831 238831
239025 239025
239366 239366
239379 239379
239550 239550
239598 239598
239606 239606
239761 239761
239784 239784
239847 239847
240021 240021
240092 240092
240118 240118
240190 240190
240252 240252
240374 240374
240486 240486
240499 240499
240558 240558
240614 240614
240629 240629
240778 240778
240959 240959
241071 241071
241177 241177
241182 241182
241263 241263
241416 241416
241668 241668
241747 241747
241812 241812
241945 241945
242039 242039
242089 242089
242103 242103
242291 242291
242368 242368
242416 242416
242569 242569
242762 242762
243023 243023
243078 243078
243200 243200
243344 243344
243358 243358
243415 243415
243425 243425
243691 243691
243763 243763
243823 243823
243856 243856
243887 243887
243998 243998
244169 244169
244394 244394
244463 244463
244483 244483
244494 244494
244513 244513
244524 244524
244693

320016 320016
320468 320468
320482 320482
320600 320601
320742 320742
320987 320987
321192 321192
321205 321205
321620 321620
322050 322050
322240 322240
322349 322349
322395 322395
322484 322484
322661 322661
322754 322754
322877 322877
323068 323068
323149 323149
323416 323416
323703 323703
323890 323890
323923 323923
323925 323925
323933 323934
324370 324370
324826 324826
324998 324998
325457 325457
325656 325656
325661 325661
325781 325781
325785 325785
326042 326042
326252 326252
327374 327374
327615 327615
327667 327667
327693 327693
327780 327780
328039 328039
328086 328086
328111 328111
328154 328154
328346 328346
328995 328995
329059 329059
329392 329392
329583 329583
329982 329982
330211 330211
330267 330267
330654 330654
330788 330788
330809 330809
331361 331361
331371 331371
331784 331784
331903 331903
332123 332123
332359 332359
332860 332860
333067 333067
333179 333179
333276 333276
333747 333747
333763 333763
333795 333795
333875 333875
333877 333877
334007 334007
334074

448181 448181
448291 448291
448568 448568
448604 448604
448672 448672
448802 448802
449002 449002
449281 449281
449298 449298
449757 449757
449794 449794
450023 450023
450224 450224
450235 450235
450416 450416
450572 450572
450820 450820
450847 450847
450929 450929
451171 451171
451456 451456
451518 451518
451721 451721
451766 451766
451791 451791
451993 451993
452103 452103
452597 452597
452973 452973
453871 453871
454077 454077
454103 454103
454280 454280
454573 454573
454664 454664
454736 454736
454803 454803
454893 454893
455309 455309
455565 455565
455621 455621
455742 455742
456377 456377
456522 456522
456829 456829
456908 456908
457303 457303
457394 457394
457831 457831
458641 458641
458678 458678
459022 459022
459559 459559
459581 459581
459675 459675
460043 460043
460206 460206
460417 460417
460530 460530
460657 460657
460846 460846
460881 460881
461076 461076
461098 461098
461499 461499
461640 461640
461655 461655
463085 463085
463298 463298
463419 463419
463439 463439
463546

688358 688358
688382 688382
688883 688883
689076 689076
689457 689457
689944 689944
689970 689970
690109 690109
690732 690732
690962 690962
691297 691297
691912 691912
691968 691968
692043 692043
693106 693106
694350 694350
697491 697491
697876 697876
698114 698114
699287 699287
700076 700076
701604 701604
702572 702572
702634 702634
702973 702973
703630 703630
703841 703841
704098 704098
705961 705961
706419 706419
706629 706629
707261 707261
707370 707370
709355 709355
709497 709497
710017 710017
710372 710372
710391 710391
710956 710956
711886 711886
713315 713315
714396 714396
714529 714529
716295 716295
716519 716519
717113 717113
717839 717839
718524 718524
719079 719079
719782 719782
720361 720361
720375 720375
721148 721148
722505 722505
722881 722881
723575 723575
724375 724375
724519 724519
724724 724724
724943 724943
725839 725839
728609 728609
728711 728711
729657 729657
730260 730260
730695 730695
731177 731177
732003 732003
732349 732349
732477 732477
732684 732684
732881

1205470 1205470
1209977 1209977
1210092 1210092
1210458 1210458
1211106 1211106
1211986 1211986
1212222 1212222
1213110 1213110
1215690 1215690
1217069 1217069
1219488 1219488
1219551 1219551
1220744 1220744
1221884 1221884
1227303 1227303
1228554 1228554
1230770 1230770
1233852 1233852
1233934 1233934
1234757 1234757
1240250 1240250
1242041 1242041
1243370 1243370
1245612 1245612
1246072 1246072
1246600 1246600
1249804 1249804
1252021 1252021
1256235 1256235
1256571 1256571
1256995 1256995
1258219 1258219
1258793 1258793
1261539 1261539
1264740 1264740
1266523 1266523
1266555 1266555
1266652 1266652
1268740 1268740
1269111 1269111
1269920 1269920
1270987 1270987
1271082 1271082
1271548 1271548
1274175 1274175
1274237 1274237
1274653 1274653
1275789 1275789
1278616 1278616
1280166 1280166
1281776 1281776
1282005 1282005
1282800 1282800
1283834 1283834
1284627 1284627
1285178 1285178
1286226 1286226
1287421 1287421
1288733 1288733
1289084 1289084
1291555 1291555
1295704 1295704
1296078 

In [67]:
freq_words = list(frequency_words_dict.keys())
print(len(freq_words))
common_words = set(words_extracted).intersection(set(freq_words))
missed_words = set(freq_words).difference(set(words_extracted))

print(len(common_words))
print(len(missed_words))
print(len(words_index))

44377
31460
12917
31460


In [71]:
#STATE_SIZE = 10
#STATE_SIZE = 300

state_size_list = [150, 100, 50 ,10]

for state_size in state_size_list:
    print(state_size)
    with open(dataset_path+'GoogleNews-vectors-negative300.txt', 'r') as read:
        with open(dataset_path+'GoogleNews-vectors-negative'+str(state_size)+'.txt', 'w') as write:
            contents = [1]
            while(len(contents) != 0):
                contents = read.readlines(30000)
                contents = [content.split()[0:state_size+1] for content in contents]
                contents = [' '.join(content)+'\n' for content in contents]
                write.writelines(contents)

150
100
50
10


In [74]:
STATE_SIZE = 300
google_mini = pd.read_csv(dataset_path+'GoogleNews-vectors-negative'+str(STATE_SIZE)+'.txt', sep =' ', header=None, prefix='col_')
#print(google_10.head())
print(google_mini.shape)
print(google_mini.columns)
print(google_mini.head())

(31462, 301)
Index(['col_0', 'col_1', 'col_2', 'col_3', 'col_4', 'col_5', 'col_6', 'col_7',
       'col_8', 'col_9',
       ...
       'col_291', 'col_292', 'col_293', 'col_294', 'col_295', 'col_296',
       'col_297', 'col_298', 'col_299', 'col_300'],
      dtype='object', length=301)
  col_0     col_1     col_2     col_3     col_4     col_5     col_6     col_7  \
0  </s>  0.001129 -0.000896  0.000319  0.001534  0.001106 -0.001404 -0.000031   
1    in  0.070312  0.086914  0.087891  0.062500  0.069336 -0.108887 -0.081543   
2   for -0.011780 -0.047363  0.044678  0.063477 -0.018188 -0.063965 -0.001312   
3  that -0.015747 -0.028320  0.083496  0.050293 -0.110352  0.031738 -0.014221   
4    is  0.007050 -0.073242  0.171875  0.022583 -0.132812  0.198242  0.112793   

      col_8     col_9  ...   col_291   col_292   col_293   col_294   col_295  \
0 -0.000420 -0.000576  ... -0.001549  0.001373 -0.000061 -0.000824  0.001328   
1 -0.154297  0.020752  ... -0.168945 -0.088867 -0.080566  0.064941

### Serialize google corpus and dictionaries to convert word into int and back

In [75]:
google_corpus_word_to_int = {}
google_corpus_int_to_word = {}

googleCorpus = set()
google_mini['col_0'].str.lower().apply(googleCorpus.add)
print(len(googleCorpus))

google_mini.index

31462


RangeIndex(start=0, stop=31462, step=1)

In [76]:
for rowIndex in google_mini.index:
    onerow = google_mini.loc[rowIndex]
    word = onerow['col_0']
    google_corpus_word_to_int[word] = rowIndex
    google_corpus_int_to_word[rowIndex] = word

outfile = dataset_path+'google_word_corpus.pic'
with open(outfile, 'wb') as pickle_file:
    pickle.dump([googleCorpus,google_corpus_word_to_int, google_corpus_int_to_word], pickle_file)

In [77]:
print(len(googleCorpus))
print(len(google_corpus_word_to_int))
print(len(google_corpus_int_to_word))

31462
31462
31462
